In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data1= pd.read_csv('historical_data.csv')

f_g_data1 = pd.read_csv('fear_greed_index.csv')

In [ ]:
print('Historical data columns :\n',data1.columns)

Historical data columns :
 Index(['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side',
       'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL',
       'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID',
       'Timestamp'],
      dtype='object')


In [ ]:
print('Fear and greed columns:\n',f_g_data1.columns)

Fear and greed columns:
 Index(['timestamp', 'value', 'classification', 'date'], dtype='object')


In [ ]:
f_g_data1['date']= pd.to_datetime(f_g_data1['date']).dt.date

In [ ]:
f_g_data1=f_g_data1.rename(columns={'classification':'sentiment',})
#rename sentiment column

In [ ]:
f_g_data1=f_g_data1[['date','sentiment','value']]

In [ ]:
f_g_data1.head()

,date,sentiment,value
0,2018-02-01,Fear,30
1,2018-02-02,Extreme Fear,15
2,2018-02-03,Fear,40
3,2018-02-04,Extreme Fear,24
4,2018-02-05,Extreme Fear,11


In [ ]:
f_g_data1.dtypes

date         object
sentiment    object
value         int64
dtype: object

In [ ]:
data1.dtypes

Account                     object
Coin                        object
Execution Price            float64
Size Tokens                float64
Size USD                   float64
Side                        object
Timestamp IST               object
Start Position             float64
Direction                   object
Closed PnL                 float64
Transaction Hash            object
Order ID                     int64
Crossed                       bool
Fee                        float64
Trade ID                   float64
date                        object
Timestamp_IST_dt    datetime64[ns]
dtype: object

In [ ]:
print(data1["Timestamp IST"].head())
print(data1["Timestamp IST"].dtype)

0    02-12-2024 22:50
1    02-12-2024 22:50
2    02-12-2024 22:50
3    02-12-2024 22:50
4    02-12-2024 22:50
Name: Timestamp IST, dtype: object
object


In [ ]:
data1["Timestamp_IST_dt"] = pd.to_datetime(data1["Timestamp IST"], dayfirst=True, errors="coerce" )

In [ ]:
data1["date"] = data1["Timestamp_IST_dt"].dt.date


Convert numeric columns

In [ ]:
data1["Closed PnL"] = pd.to_numeric(data1["Closed PnL"], errors="coerce")
data1["Fee"] = pd.to_numeric(data1["Fee"], errors="coerce")

In [ ]:
data1["Size USD"] = pd.to_numeric(data1["Size USD"], errors="coerce")

MERGE DATASETS

In [ ]:
merged_df = data1.merge(f_g_data1, on="date",
    how="left"
)

In [ ]:
merged_df = merged_df.dropna(subset=["sentiment"])

In [ ]:
print('Merged columns :',merge_df.columns)

Merged columns : Index(['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side',
       'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL',
       'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID',
       'Timestamp', 'date', 'sentiment', 'value'],
      dtype='object')


In [ ]:
print(f_g_data1["date"].min())
print(f_g_data1["date"].max())
print(f_g_data1["date"].nunique())


2018-02-01
2025-05-02
2644


In [ ]:
print(data1["date"].min())
print(data1["date"].max())
print(data1["date"].nunique())


2023-05-01
2025-05-01
480


The dataset is time-aligned correctly

No structural bias from missing sentiment dates

All downstream analysis is valid

FEATURE ENGINEERING

In [ ]:
#Net Pnl
merged_df["net_pnl"] = merged_df["Closed PnL"] - merged_df["Fee"]

#Win / Loss flag
merged_df["is_profit"] = (merged_df["net_pnl"] > 0).astype(int)

#Absolute Pnl
merged_df["abs_pnl"] = merged_df["net_pnl"].abs()

#Return percentage
merged_df = merged_df[merged_df["Size USD"] > 0]

merged_df["return_pct"] = merged_df["net_pnl"] / merged_df["Size USD"]


1 → profitable trade

0 → losing or breakeven trade

BASIC PERFORMANCE SUMMARY

In [ ]:
performance_summary = merged_df.groupby("sentiment").agg(
    number_of_trades = ("Account", "count"),
    total_net_pnl = ("net_pnl", "sum"),
    avg_net_pnl = ("net_pnl", "mean"),
    win_rate = ("is_profit", "mean"),
    avg_trade_size = ("Size USD", "mean"),
    total_volume = ("Size USD", "sum"),
    total_fees = ("Fee", "sum"),
    pnl_volatility = ("net_pnl", "std"),
    avg_return_pct = ("return_pct", "mean")
)

performance_summary["win_rate"] * 100
performance_summary["avg_return_pct"] * 100

performance_summary.round(2)

,number_of_trades,total_net_pnl,avg_net_pnl,win_rate,avg_trade_size,total_volume,total_fees,pnl_volatility,avg_return_pct
sentiment,,,,,,,,,
Extreme Fear,21315,715222.25,33.55,0.37,5371.05,1.144838e+08,23888.51,1136.94,0.00
Extreme Greed,39729,2688063.79,67.66,0.47,3132.83,1.244642e+08,27030.29,769.04,0.04
Fear,61586,3264684.32,53.01,0.41,7847.95,4.833240e+08,92456.52,936.54,0.01
Greed,50022,2086997.29,41.72,0.39,5769.09,2.885815e+08,63098.48,1119.27,0.02
Neutral,37496,1253697.20,33.44,0.40,4806.95,1.802414e+08,39373.41,516.98,0.01


PnL Volatility (Std Dev)



Risk spikes not only during Greed, but also during Extreme Fear.
Extreme Fear is associated with panic-driven, unstable outcomes
Greed volatility is high, but Extreme Fear is worse

Average Trade Size

Fear trades are the largest on average,
Extreme Greed trades are the smallest

Win Rate Patterns (Decision Quality)

Extreme Greed has the highest win rate
, Extreme Fear has the lowest win rate

This is crucial:

Extreme Greed → many small wins
, Extreme Fear → many losing trades

Average Net PnL (Trade Quality)

Extreme Greed has the highest average PnL
, Neutral and Extreme Fear are weakest

Volume & Overtrading Signal :

Fear dominates capital deployment
, Greed dominates trade count, not size

BEHAVIOR SUMMARY


In [ ]:
behavior_summary = merged_df.groupby("sentiment").agg(
    win_rate=("is_profit", "mean"),
    avg_return_pct=("return_pct", "mean")
)

behavior_summary["win_rate"] *= 100
behavior_summary["avg_return_pct"] *= 100

behavior_summary.round(2)


,win_rate,avg_return_pct
sentiment,,
Extreme Fear,36.84,0.41
Extreme Greed,46.79,3.96
Fear,41.14,1.49
Greed,39.16,1.93
Neutral,39.58,1.04


RISK BEHAVIOR ANALYSIS

In [ ]:
risk_summary = merged_df.groupby("sentiment").agg(
    pnl_volatility = ("net_pnl", "std"),
    max_loss = ("net_pnl", "min"),
    avg_loss = ("net_pnl", lambda x: x[x < 0].mean() if (x < 0).any() else 0),
    avg_abs_pnl = ("abs_pnl", "mean"),
    downside_volatility=("net_pnl", lambda x: x[x < 0].std())
)

if "leverage" in merged_df.columns:
    risk_summary["avg_leverage"] = merged_df.groupby("sentiment")["leverage"].mean()


risk_summary.round(2)

,pnl_volatility,max_loss,avg_loss,avg_abs_pnl,downside_volatility
sentiment,,,,,
Extreme Fear,1136.94,-31044.72,-56.08,94.57,559.15
Extreme Greed,769.04,-10268.23,-13.87,82.07,223.01
Fear,936.54,-35705.50,-18.68,74.02,385.04
Greed,1119.27,-118071.56,-36.34,85.65,983.44
Neutral,516.98,-24510.75,-18.17,55.28,286.89


Fear changes how traders trade; Greed changes how traders lose.

Overtrading (daily trades)

In [ ]:
daily_trades = (
    merged_df.groupby(["date", "sentiment"]).size()
    .reset_index(name="daily_trade_count")
)

daily_avg_trades = daily_trades.groupby("sentiment")["daily_trade_count"].mean()
daily_avg_trades

sentiment
Extreme Fear     1522.500000
Extreme Greed     348.500000
Fear              707.885057
Greed             259.181347
Neutral           559.641791
Name: daily_trade_count, dtype: float64

Extreme Fear exhibits the highest daily trading activity, indicating panic-driven overtrading. Fear and Neutral show moderate, more controlled participation. Greed and Extreme Greed have the lowest activity levels, suggesting fewer but more selective or payoff-oriented trades rather than reactive behavior.

Long / Short bias

In [ ]:
long_short_bias = pd.crosstab(
    merged_df["sentiment"],
    merged_df["Side"],
    normalize="index"
) * 100
long_short_bias.round(2)

Side,BUY,SELL
sentiment,,
Extreme Fear,51.13,48.87
Extreme Greed,44.89,55.11
Fear,48.99,51.01
Greed,48.94,51.06
Neutral,50.37,49.63


Directional bias is mostly balanced across sentiments. Extreme Fear shows a slight BUY bias, while Extreme Greed shows a clear SELL bias, suggesting profit-taking at euphoric levels. Other regimes remain close to neutral with no strong directional preference.

Account-level adaptability

In [ ]:
account_perf = (
    merged_df.groupby(["Account", "sentiment"])["net_pnl"]
    .sum()
    .unstack()
)

account_perf.head()


sentiment,Extreme Fear,Extreme Greed,Fear,Greed,Neutral
Account,,,,,
0x083384f897ee0f19899168e3b1bec365f52a9012,124625.229446,-42078.309495,1.110127e+06,275800.917329,124348.968632
0x23e7a7f8d14b550961925fbfdaa92f5d195ba5bd,-4298.443493,35216.661056,-2.234960e+03,28667.627489,-12193.561940
0x271b280974205ca63b716753467d5a371de622ab,15525.749777,1758.498164,2.620507e+04,-102780.096282,-20426.394111
0x28736f43f1e871e6aa8b1148d38d4994275d72c4,-25820.191470,106821.333371,1.315235e+04,30710.767221,5334.102872
0x2c229d22b100a7beb69122eed721cee9b24011dd,11323.139636,89362.693753,3.807315e+03,19741.859273,41313.297571


Statistical Inference

Split datasets

In [ ]:
fear = merged_df[merged_df["sentiment"] == "Fear"]
greed = merged_df[merged_df["sentiment"] == "Greed"]


Mann–Whitney U (returns)

In [ ]:
from scipy.stats import mannwhitneyu

u_stat, p_value = mannwhitneyu(
    fear["return_pct"],
    greed["return_pct"],
    alternative="two-sided"
)

u_stat, p_value


(1642546826.5, 2.691888096721902e-81)

Effect Size (Rank-Biserial)

In [ ]:
n1 = len(fear)
n2 = len(greed)

rank_biserial = 1 - (2 * u_stat) / (n1 * n2)

print("Rank-biserial effect size:", rank_biserial)


Rank-biserial effect size: -0.06636211070928355


In [ ]:
fear["return_pct"].describe(), greed["return_pct"].describe()


(count    61586.000000
 mean         0.014919
 std          0.074292
 min         -1.432547
 25%         -0.000300
 50%         -0.000048
 75%          0.007170
 max          2.604034
 Name: return_pct, dtype: float64,
 count    50022.000000
 mean         0.019296
 std          0.098679
 min         -3.101267
 25%         -0.000350
 50%         -0.000050
 75%          0.010395
 max          1.487605
 Name: return_pct, dtype: float64)

Binary test Z-test for win rate

In [ ]:
from statsmodels.stats.proportion import proportions_ztest

successes = [
    fear["is_profit"].sum(),
    greed["is_profit"].sum()
]

trials = [
    len(fear),
    len(greed)
]

z_stat, p_win = proportions_ztest(successes, trials)

z_stat, p_win


(6.705428222327828, 2.0081658482341934e-11)

Bootstrap Confidence Interval (Mean Return Diff)

In [ ]:
import numpy as np

def bootstrap_mean_diff(x, y, n_boot=5000):
    diffs = []
    for _ in range(n_boot):
        x_s = np.random.choice(x, size=len(x), replace=True)
        y_s = np.random.choice(y, size=len(y), replace=True)
        diffs.append(x_s.mean() - y_s.mean())
    return np.percentile(diffs, [2.5, 97.5])

ci = bootstrap_mean_diff(
    fear["return_pct"].values,
    greed["return_pct"].values
)

ci


array([-0.00541253, -0.00331571])

Statistical tests confirm that Fear and Greed trades come from significantly different return distributions (Mann–Whitney p ≪ 0.001). However, the effect size is small (rank-biserial = −0.066), indicating that Greed trades only marginally outperform Fear trades on average. Win-rate analysis shows Fear trades are significantly more accurate (z = 6.71, p ≪ 0.001), while bootstrap confidence intervals reveal a small but stable Greed advantage of roughly 0.3–0.5% per trade. Overall, sentiment-driven differences are statistically robust but economically modest, reflecting behavioral asymmetry rather than a strong tradable edge.

tests

In [ ]:
avg_trade_size = merged_df.groupby("sentiment")["Size USD"].mean()
avg_trade_size

sentiment
Extreme Fear     5371.045974
Extreme Greed    3132.830817
Fear             7847.952199
Greed            5769.092425
Neutral          4806.950315
Name: Size USD, dtype: float64

In [ ]:
account_sentiment_perf = (
    merged_df.groupby(["Account", "sentiment"])["net_pnl"]
    .sum()
    .unstack()
)


In [ ]:
trade_count = merged_df.groupby("sentiment")["Account"].count()
trade_count

sentiment
Extreme Fear     21315
Extreme Greed    39729
Fear             61586
Greed            50022
Neutral          37496
Name: Account, dtype: int64

In [ ]:
total_volume = merged_df.groupby("sentiment")["Size USD"].sum()
total_volume

sentiment
Extreme Fear     1.144838e+08
Extreme Greed    1.244642e+08
Fear             4.833240e+08
Greed            2.885815e+08
Neutral          1.802414e+08
Name: Size USD, dtype: float64

In [ ]:
max_loss = merged_df.groupby("sentiment")["net_pnl"].min()
max_loss

sentiment
Extreme Fear     -31044.717218
Extreme Greed    -10268.229064
Fear             -35705.501042
Greed           -118071.556516
Neutral          -24510.750000
Name: net_pnl, dtype: float64

In [ ]:
filtered_df = merged_df[merged_df["Size USD"] > 10]

In [ ]:
print(
    merged_df[
        ["Closed PnL", "Fee", "net_pnl", "is_profit", "return_pct"]
    ].head()
)

print(merged_df[["net_pnl", "abs_pnl", "return_pct"]].describe())


   Closed PnL       Fee   net_pnl  is_profit  return_pct
0         0.0  0.345404 -0.345404          0   -0.000044
1         0.0  0.005600 -0.005600          0   -0.000044
2         0.0  0.050431 -0.050431          0   -0.000044
3         0.0  0.050043 -0.050043          0   -0.000044
4         0.0  0.003055 -0.003055          0   -0.000044
             net_pnl        abs_pnl     return_pct
count  211175.000000  211175.000000  211175.000000
mean       47.394993      76.679100       0.018632
std       917.539276     915.557156       0.845878
min   -118071.556516       0.000000    -384.406762
25%        -0.194161       0.062490      -0.000300
50%        -0.005860       0.783071      -0.000048
75%         5.539295      11.866363       0.010105
max    135299.803088  135299.803088       3.403500


In [ ]:
summary["median_net_pnl"] = merged_df.groupby("sentiment")["net_pnl"].median()
summary["median_return_pct"] = merged_df.groupby("sentiment")["return_pct"].median() * 100

In [ ]:
risk_volatility = merged_df.groupby("sentiment")["net_pnl"].std()
risk_volatility

sentiment
Extreme Fear     1136.936709
Extreme Greed     769.042255
Fear              936.537683
Greed            1119.268782
Neutral           516.980038
Name: net_pnl, dtype: float64